**MODELO PARA LA GENERACION DE DATOS ALEATORIOS PARA TIEMPOS DEL PROCESO DE CAMBIO DE ACEITE EN CARROS**

In [3]:
import math
import random
import time

# === Generador base (producto constante) ===
def multiplicador_constante(semilla, a, d, n):
    numeros = []
    x = semilla
    for _ in range(n):
        producto = x * a
        producto_str = str(producto).zfill(2 * d)
        inicio = (len(producto_str) - d) // 2
        fin = inicio + d
        x = int(producto_str[inicio:fin])
        numeros.append(x)
    return numeros

# Normalizar a [0,1]
def uniformes(semilla, a, d, n):
    base = multiplicador_constante(semilla, a, d, n)
    return [x / (10**d - 1) for x in base]


# === Distribuciones ===
def poisson(semilla, a, d, n, lam):
    resultados = []
    for _ in range(n):
        L = math.exp(-lam)
        k, p = 0, 1
        while p > L:
            k += 1
            p *= random.random()
        resultados.append(k - 1)
    return resultados

def exponencial(semilla, a, d, n, lam):
    u = uniformes(semilla, a, d, n)
    return [-(1/lam) * math.log(1 - ui) for ui in u]

def normal(semilla, a, d, n, mu, sigma):
    u = uniformes(semilla, a, d, 2*n)
    resultados = []
    for i in range(0, len(u), 2):
        z = math.sqrt(-2 * math.log(u[i])) * math.cos(2 * math.pi * u[i+1])
        resultados.append(mu + sigma * z)
    return resultados[:n]


# =================== SIMULACIÓN ===================
def simulacion():
    # Inputs fijos del generador
    a = 3761
    d = 4

    # Cada ejecución usa la hora actual como semilla
    semilla = int(time.time()) % 10000  

    # Usuario selecciona cantidad de datos
    n = int(input("Ingrese la cantidad de horas/días a simular: "))

    # Generación de datos
    llegadas = poisson(semilla, a, d, n, lam=3)            # Cantidad de vehículos
    entre_llegadas = exponencial(semilla, a, d, n, lam=0.5) # Tiempos entre llegadas
    servicio = normal(semilla, a, d, n, mu=20, sigma=3)     # Tiempo de servicio

    # Mostrar resultados
    print("\n=== RESULTADOS DE LA SIMULACIÓN ===")
    print("Llegadas):", llegadas) #(Poisson, λ=3)
    print("Horas entre llegadas:", [round(x,2) for x in entre_llegadas]) #(Exponencial, λ=0.5)
    print("Minutos de servicio ):", [round(x,2) for x in servicio]) #(Normal μ=20, σ=3


# Ejecutar simulación
if __name__ == "__main__":
    simulacion()



=== RESULTADOS DE LA SIMULACIÓN ===
Llegadas): [2, 1, 2, 4, 2, 2, 1, 3, 2, 3, 4, 3, 3, 4, 2, 6, 1, 3, 2, 1, 1, 1, 1, 2, 7, 4, 3, 2, 2, 3, 2, 6, 2, 1, 3, 1, 3, 0, 2, 1, 3, 2, 3, 4, 6, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 2, 2, 3, 0, 6, 7, 3, 4, 3, 2, 2, 2, 1, 3, 6, 2, 5, 0, 4, 0, 4, 2, 3, 1, 1, 3, 4, 0, 3, 3, 3, 5, 2, 8, 4, 3, 2, 7, 5, 4, 4, 5, 5, 3]
Horas entre llegadas: [0.71, 0.48, 6.18, 4.58, 3.22, 0.19, 1.19, 4.23, 0.15, 2.09, 0.89, 1.54, 0.33, 2.37, 0.22, 6.11, 3.6, 1.01, 5.15, 2.67, 2.41, 0.76, 4.13, 3.78, 4.86, 0.71, 0.7, 0.13, 1.15, 1.16, 1.48, 2.0, 3.07, 1.41, 9.95, 0.85, 0.08, 1.3, 7.6, 2.89, 2.57, 0.4, 3.18, 5.67, 1.03, 0.27, 2.9, 2.99, 0.36, 0.25, 1.31, 0.05, 9.92, 0.84, 4.54, 2.5, 3.65, 1.52, 0.03, 1.84, 1.99, 2.35, 7.0, 1.27, 2.35, 0.03, 1.63, 5.43, 0.25, 0.95, 0.47, 3.94, 0.89, 1.49, 2.59, 0.77, 0.06, 0.4, 3.26, 0.51, 1.02, 7.83, 3.9, 0.61, 3.75, 3.49, 0.1, 2.92, 4.07, 2.4, 0.59, 1.88, 4.95, 1.16, 1.49, 2.99, 0.35, 7.82, 3.85, 0.25]
Minutos de servicio ): [21.12, 20.74, 21.